Matrices must be in a rebinned sparse graal format in a Rebin/ directory (generated with the master Script_hicstuff_pipeline.sh) and in dense format binned at 1 kb (generated using HiC_Sparse_to_dense_fromHicstuff_DpnIIHinfI.ipynb) in a Dense/ directory, located in the main input directory (input_dir). 

In [1]:
import numpy as np
import sys
import pandas as pd
from scipy import stats

The output directory must contain a Intra/, Centro/, and Telo/ directory

In [2]:
dir_seqs= "/mnt/d/Science/OwnCloud/Sequences/Genomes/"
out= "/mnt/d/Science/OwnCloud/Experiments/HiC/Quantif/"
input_dir = "/mnt/e/Science/HiC/Contact_files/"

#specify the path to the tab-delimited file containing chromosome name, length, and centromere location
chrs = pd.read_csv(str(dir_seqs + "Yeast_genome_S288c_R64-2-1_2014/S288c_chr_centro_coordinates.tsv"), sep = "\t")

Specify the number of bins (in kb) to consider around the centromere of interest at the "extra_bin" argument 

In [4]:
samples = ["AD280_S288c_DSB_cutsite_q20"]
for sample in samples:
    frags = pd.read_csv(str(input_dir + "Rebin/" + sample + "_1kb.frags.tsv"), sep = "\t")
    mat = pd.read_csv(str(input_dir + "Dense/" + sample + "_dense_1kb.mat.csv"), sep = " ", header = None)
    mat = np.array(mat)
    print(mat)
    
    print("total number of contacts in the matrix =", sum(sum(mat))/2)
    mat_norm=mat/sum(sum(mat))


    #Define the lines in frags corresponding to each chromosomes in chrs.
    #Define the lines in frags of start and end positions of fragments corresponding to centromeres [-X kb to +X kb] around the cen-containing bin (= 5 bins total)
    #Define the lines in frags of the start and end positions of each telomere [consider a whole 20kb region]
    #chromosomes
    coord_chr_start = []
    coord_chr_end = []
    i = 0
    for i in chrs.loc[: ,"Chr"]:
        coord_chr_start.append(min(frags.loc[frags["chrom"]== str(i)].index.values))
        coord_chr_end.append(max(frags.loc[frags["chrom"]== str(i)].index.values))
    #print(coord_chr_start)
    #print(coord_chr_end)

    #centromeres
    #extra_bin = input("extra kb on each side of the centromeric bin?") #Normally 2 to have a 5kb window
    extra_bin = 14
    coord_cen_start = []
    coord_cen_end = []
    i = 0
    for i in chrs.loc[: ,"Chr"]:
        cen = int(chrs.loc[chrs["Chr"] == str(i)]["Left_arm_length"])
        cen_row=int(frags.loc[(frags["chrom"]== str(i)) & (frags["start_pos"] < cen) & (frags["end_pos"] > cen)].index.values)
        coord_cen_start.append(cen_row - int(extra_bin))
        coord_cen_end.append(cen_row + int(extra_bin) + 1)
    #print(coord_cen_start)
    #print(coord_cen_end)


    print("--------------------------")
    #Quantif telomeres contacts (20 kb window from telomeric ends) 
    coord_tel_start = []
    coord_tel_end = []
    i = 0
    for i in chrs.loc[: ,"Chr"]:
        tel1 = 1+2
        tel2 = int(chrs.loc[chrs["Chr"] == str(i)]["Length"]) -1000
        tel1_row=int(frags.loc[(frags["chrom"]== str(i)) & (frags["start_pos"] < tel1) & (frags["end_pos"] > tel1)].index.values)
        coord_tel_start.append(tel1_row)
        coord_tel_end.append(tel1_row + 19)
        tel2_row=int(frags.loc[(frags["chrom"]== str(i)) & (frags["start_pos"] < tel2) & (frags["end_pos"] > tel2)].index.values)
        coord_tel_start.append(tel2_row - 19)
        coord_tel_end.append(tel2_row)
    #print(coord_tel_start)
    #print(coord_tel_end)
    print("done")


    #QUANTIF INTRA CONTACTS FOR EACH CHROMOSOMES
    print("--------------------------")
    output_dir = out + "Intra/"
    intra_contacts = []
    i=0
    while i < len(coord_chr_start):
        start = coord_chr_start[i]
        end = coord_chr_end[i]
        intra=0
        intra=sum(sum(mat_norm[start : end , start : end]))
        all_contacts=sum(sum(mat_norm[start : end , ]))
        intra_contacts.append(intra/all_contacts)
        i += 1
    print(intra_contacts)
    print("done")
    np.savetxt(output_dir + str(sample) + "_intra.tsv", intra_contacts, delimiter = "\t")



    print("--------------------------")
    #Quantify the contacts between the centromeres
    output_dir = out + "Centro/"
    cen_contacts = []
    i=0
    while i < len(coord_cen_start):
        j=0
        while j < len(coord_cen_start):
            if i != j:
                start1 = coord_cen_start[i]
                end1 = coord_cen_end[i]
                start2 = coord_cen_start[j]
                end2 = coord_cen_end[j]
                cen_contacts.append(sum(sum(mat_norm[start1 : end1 , start2 : end2])))
            j += 1
        i += 1
    #print(cen_contacts)
    print("done")
    np.savetxt(output_dir + str(sample) + "_centro_binsize_" + str(int(extra_bin) * 2 + 1) + "kb.tsv", cen_contacts, delimiter = "\t")



    print("--------------------------")
    #Quantify the contacts between CEN5 and other 15 centromeres
    output_dir = out + "Centro/"
    cen_contacts_chr5 = []
    i = 4
    j = 0
    while j < len(coord_cen_start):
        if i != j:
            start1 = coord_cen_start[i]
            end1 = coord_cen_end[i]
            start2 = coord_cen_start[j]
            end2 = coord_cen_end[j]
            cen_contacts_chr5.append(sum(sum(mat_norm[start1 : end1 , start2 : end2])))
        j += 1

    print(cen_contacts_chr5)
    print("done")
    np.savetxt(output_dir + str(sample) + "_chr5_centro_binsize_" + str(int(extra_bin) * 2 + 1) + "kb.tsv", cen_contacts_chr5, delimiter = "\t")

    print("--------------------------")
    #Quantify contcts between telomeres
    output_dir = out + "Telo/"
    tel_contacts = []
    i=0
    while i < len(coord_tel_start):
        j=0
        while j < len(coord_tel_end):
            if i != j:
                a = 0
                start1 = coord_tel_start[i]
                end1 = coord_tel_end[i]
                start2 = coord_tel_start[j]
                end2 = coord_tel_end[j]
                a = sum(sum(mat_norm[start1 : end1 , start2 : end2]))
                if a != 0:
                    tel_contacts.append(a)
            j += 1
        i += 1
    #print(tel_contacts)
    print("done")
    np.savetxt(output_dir + str(sample) + "_telo_20kb.tsv", tel_contacts, delimiter = "\t")



[[ 49 246 112 ...   0   0   0]
 [246 120 143 ...   0   0   0]
 [112 143   0 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
total number of contacts in the matrix = 36355011.0


/tmp/ipykernel_1062/3182205072.py:73: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  cen = int(chrs.loc[chrs["Chr"] == str(i)]["Left_arm_length"])
/tmp/ipykernel_1062/3182205072.py:74: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cen_row=int(frags.loc[(frags["chrom"]== str(i)) & (frags["start_pos"] < cen) & (frags["end_pos"] > cen)].index.values)
/tmp/ipykernel_1062/3182205072.py:88: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  tel2 = int(chrs.loc[chrs["Chr"] == str(i)]["Length"]) -1000
/tmp/ipykernel_1062/3182205072.py:89: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in 

--------------------------
done
--------------------------
[0.7078253643126919, 0.7853107955359898, 0.7395969588343048, 0.8127713657543134, 0.7536503705048677, 0.7268655983138133, 0.7907977995218648, 0.7565976939506907, 0.7412754120214778, 0.7773770064334908, 0.7646407475848325, 0.8961839226781791, 0.7832006105129857, 0.7840031984011891, 0.7861299204684785, 0.7802978600117827]
done
--------------------------
done
--------------------------
[1.2873053456097154e-05, 1.0645024973311109e-05, 1.6008797246684922e-05, 1.2721767571463533e-05, 1.189657183709833e-05, 1.5719978739657103e-05, 1.583000483757246e-05, 1.7164071274796205e-05, 1.4991055840967837e-05, 1.873194317009009e-05, 1.5183601512319717e-05, 1.3835781812856556e-05, 1.51698482500803e-05, 1.7136564750317363e-05, 1.526612108575624e-05]
done
--------------------------
[8.637048686355781e-06, 1.0094894483734307e-05, 8.650801948595203e-06, 1.0645024973311109e-05, 8.609542161876944e-06, 1.0328699941804445e-05, 1.0108647745973726e-05, 1.2

/tmp/ipykernel_1062/3182205072.py:73: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  cen = int(chrs.loc[chrs["Chr"] == str(i)]["Left_arm_length"])
/tmp/ipykernel_1062/3182205072.py:74: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cen_row=int(frags.loc[(frags["chrom"]== str(i)) & (frags["start_pos"] < cen) & (frags["end_pos"] > cen)].index.values)
/tmp/ipykernel_1062/3182205072.py:88: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  tel2 = int(chrs.loc[chrs["Chr"] == str(i)]["Length"]) -1000
/tmp/ipykernel_1062/3182205072.py:89: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in 

--------------------------
done
--------------------------
[0.596956182545608, 0.6795239918971953, 0.6298588713436146, 0.711808508423639, 0.6443378493110287, 0.6229371711812709, 0.6861087778430269, 0.651877390677841, 0.6339666786165703, 0.6661755513657719, 0.6574964014418756, 0.8164472147759383, 0.6739265175660181, 0.6791175572562527, 0.6811959222897408, 0.6729055103777306]
done
--------------------------
done
--------------------------
[1.355747237883572e-05, 1.1439502125284636e-05, 1.4973557141384411e-05, 1.284327310728942e-05, 1.2707821521306508e-05, 1.5675442632386806e-05, 1.4382495675277134e-05, 1.678368288133796e-05, 1.48873606775771e-05, 1.8076629838447626e-05, 1.4985870921928315e-05, 1.494892958029661e-05, 1.533065677715756e-05, 1.6180307634686775e-05, 1.498587092192831e-05]
done
--------------------------
[9.580121263155485e-06, 1.1033147367335881e-05, 9.481611018804273e-06, 1.1439502125284633e-05, 9.555493702067684e-06, 1.1008519806248077e-05, 1.0762244195370045e-05, 1.178428

...

--------------------------
done
--------------------------
[0.6084937487153376, 0.6616720426311462, 0.6184477778385453, 0.7008792489876021, 0.6226433328246328, 0.6238225490867468, 0.6707727280977476, 0.6350719967005436, 0.6248423157091296, 0.6587842428751874, 0.6521157409796938, 0.7782271395930004, 0.6637712476803281, 0.6605576381204585, 0.6687697664469653, 0.6617265161656709]
done
--------------------------
done
--------------------------
[1.1062026375569585e-05, 1.0593296444401382e-05, 1.1147249999418347e-05, 1.0678520068250146e-05, 1.0286491398545835e-05, 1.3132960435094552e-05, 1.14540550452739e-05, 1.3610212728647631e-05, 1.2681275228696102e-05, 1.4752209288221075e-05, 1.3448287843334983e-05, 1.283467775162388e-05, 1.2391514907610306e-05, 1.2953990825012152e-05, 1.2272201834222038e-05]
done
--------------------------
[8.215557339020862e-06, 8.599063646340302e-06, 8.982569953659738e-06, 1.0593296444401385e-05, 8.283736238099874e-06, 9.851850916917132e-06, 9.28937499951529e-06, 1.05

/tmp/ipykernel_1062/3182205072.py:73: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  cen = int(chrs.loc[chrs["Chr"] == str(i)]["Left_arm_length"])
/tmp/ipykernel_1062/3182205072.py:74: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cen_row=int(frags.loc[(frags["chrom"]== str(i)) & (frags["start_pos"] < cen) & (frags["end_pos"] > cen)].index.values)
/tmp/ipykernel_1062/3182205072.py:88: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  tel2 = int(chrs.loc[chrs["Chr"] == str(i)]["Length"]) -1000
/tmp/ipykernel_1062/3182205072.py:89: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in 

--------------------------
done
--------------------------
[0.5735867098067277, 0.6264631039795353, 0.5921421875920692, 0.66583063736217, 0.5975893073046209, 0.5951547380357577, 0.6395786370510693, 0.6086545261938034, 0.5988123039309607, 0.6250781028073347, 0.6177203428927673, 0.7355709506554726, 0.629538211718597, 0.629321926631548, 0.6338734511980588, 0.6296716028497206]
done
--------------------------
done
--------------------------
[1.4296999029166478e-05, 1.2195232175953741e-05, 1.4596064668358803e-05, 1.3881630085843804e-05, 1.2893052000735832e-05, 1.5285577114274443e-05, 1.468744583588979e-05, 1.7046741433962585e-05, 1.4712367972489153e-05, 1.8392536810328054e-05, 1.6141237137519155e-05, 1.4978204096215661e-05, 1.4529605637427175e-05, 1.7055048812829045e-05, 1.5077892642613106e-05]
done
--------------------------
[1.0998969619184436e-05, 1.0434067856265595e-05, 1.05586785392624e-05, 1.219523217595374e-05, 9.51194880208926e-06, 1.1771555853764612e-05, 1.1314650016109669e-05, 1.35

/tmp/ipykernel_1062/3182205072.py:73: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  cen = int(chrs.loc[chrs["Chr"] == str(i)]["Left_arm_length"])
/tmp/ipykernel_1062/3182205072.py:74: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cen_row=int(frags.loc[(frags["chrom"]== str(i)) & (frags["start_pos"] < cen) & (frags["end_pos"] > cen)].index.values)
/tmp/ipykernel_1062/3182205072.py:88: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  tel2 = int(chrs.loc[chrs["Chr"] == str(i)]["Length"]) -1000
/tmp/ipykernel_1062/3182205072.py:89: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in 

--------------------------
done
--------------------------
[0.5703490874833061, 0.6195169130014467, 0.5839255245672036, 0.6590750673303237, 0.584354544293967, 0.5894541858775988, 0.6303461527982561, 0.6014046776064746, 0.5914849265330511, 0.6194306279096805, 0.6128592118330903, 0.7541974861347244, 0.6215743096875649, 0.62166314152786, 0.6269033231678026, 0.6254276724174236]
done
--------------------------
done
--------------------------
[1.1776480662358614e-05, 1.111152389828108e-05, 1.2287244553606577e-05, 1.0889871643588572e-05, 1.0475478297859097e-05, 1.2585993244713877e-05, 1.1535554298562408e-05, 1.4667597027913105e-05, 1.2788371390302687e-05, 1.5004893937227792e-05, 1.2576356190162022e-05, 1.2981112481339652e-05, 1.2557082081058329e-05, 1.2798008444854535e-05, 1.3010023644995195e-05]
done
--------------------------
[9.550321060881658e-06, 9.126290660600335e-06, 9.280483533429909e-06, 1.1111523898281083e-05, 8.557704442041286e-06, 1.0677856443447912e-05, 9.916529133851894e-06, 1.1